In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn
import pickle


# PJME Energy Model

Notebook to train the prediction model for PJM East Region Hourly Consumption.

A broader Exploratory Data Analysis (EDA) and Model Study is available at my [Energy_Consumption-Time_Series_Forecasting_Study](https://github.com/diegokurashima/Energy_Consumption-Time_Series_Forecasting_Study) repository.

## Import Dataset

In [2]:
df_PJME_raw = pd.read_csv("PJME_hourly.csv")

df_PJME = df_PJME_raw.copy()
df_PJME['Datetime'] = pd.to_datetime(df_PJME['Datetime'])

df_PJME = df_PJME.set_index("Datetime")
df_PJME = df_PJME.sort_index()

print("PJME Data Info:")
print(df_PJME.info())
print("\n")

PJME Data Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145366 entries, 2002-01-01 01:00:00 to 2018-08-03 00:00:00
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   PJME_MW  145366 non-null  float64
dtypes: float64(1)
memory usage: 2.2 MB
None




### Datetime: Missing and Duplicates

In [3]:
missing_dates = pd.date_range(start=pd.to_datetime('2002-01-01 01:00:00'), end=pd.to_datetime('2018-08-03 00:00:00'), freq='h').difference(df_PJME.index)
missing_dates

DatetimeIndex(['2002-04-07 03:00:00', '2002-10-27 02:00:00',
               '2003-04-06 03:00:00', '2003-10-26 02:00:00',
               '2004-04-04 03:00:00', '2004-10-31 02:00:00',
               '2005-04-03 03:00:00', '2005-10-30 02:00:00',
               '2006-04-02 03:00:00', '2006-10-29 02:00:00',
               '2007-03-11 03:00:00', '2007-11-04 02:00:00',
               '2008-03-09 03:00:00', '2008-11-02 02:00:00',
               '2009-03-08 03:00:00', '2009-11-01 02:00:00',
               '2010-03-14 03:00:00', '2010-11-07 02:00:00',
               '2010-12-10 00:00:00', '2011-03-13 03:00:00',
               '2011-11-06 02:00:00', '2012-03-11 03:00:00',
               '2012-11-04 02:00:00', '2013-03-10 03:00:00',
               '2013-11-03 02:00:00', '2014-03-09 03:00:00',
               '2015-03-08 03:00:00', '2016-03-13 03:00:00',
               '2017-03-12 03:00:00', '2018-03-11 03:00:00'],
              dtype='datetime64[ns]', freq=None)

In [4]:
df_PJME = df_PJME.resample("1h").mean()

missing_dates = pd.date_range(start=pd.to_datetime('2002-01-01 01:00:00'), end=pd.to_datetime('2018-08-03 00:00:00'), freq='h').difference(df_PJME.index)
missing_dates

DatetimeIndex([], dtype='datetime64[ns]', freq='h')

## Feature Engineering

### Datetime features

In [5]:
def create_features_Datetime(df_input):
  '''
  Create datetime features for "Datetime" column
  '''
  
  df = df_input.copy()

  # Standard Datetime Features
  df['Year'] = df.index.year
  df['Quarter'] = df.index.quarter
  df['Month'] = df.index.month
  df['Day'] = df.index.day
  df['Hour'] = df.index.hour

  df['Day_of_Year'] = df.index.dayofyear

  df['Weekday'] = df.index.weekday
  df['Is_Weekend'] =  df.index.day_name().isin(['Saturday', 'Sunday'])

  return df

df_PJME = create_features_Datetime(df_PJME)

In [6]:
df_PJME.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145392 entries, 2002-01-01 01:00:00 to 2018-08-03 00:00:00
Freq: h
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   PJME_MW      145362 non-null  float64
 1   Year         145392 non-null  int32  
 2   Quarter      145392 non-null  int32  
 3   Month        145392 non-null  int32  
 4   Day          145392 non-null  int32  
 5   Hour         145392 non-null  int32  
 6   Day_of_Year  145392 non-null  int32  
 7   Weekday      145392 non-null  int32  
 8   Is_Weekend   145392 non-null  bool   
dtypes: bool(1), float64(1), int32(7)
memory usage: 6.2 MB


### Final Processing

Drop NA

In [7]:
df_PJME = df_PJME.dropna()
df_PJME.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145362 entries, 2002-01-01 01:00:00 to 2018-08-03 00:00:00
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   PJME_MW      145362 non-null  float64
 1   Year         145362 non-null  int32  
 2   Quarter      145362 non-null  int32  
 3   Month        145362 non-null  int32  
 4   Day          145362 non-null  int32  
 5   Hour         145362 non-null  int32  
 6   Day_of_Year  145362 non-null  int32  
 7   Weekday      145362 non-null  int32  
 8   Is_Weekend   145362 non-null  bool   
dtypes: bool(1), float64(1), int32(7)
memory usage: 6.2 MB


## Training Model

In [8]:
# Define Train and Target Columns
cols_train = ['Quarter', 'Month', 'Day', 'Hour', 'Day_of_Year', 'Weekday', 'Is_Weekend']
cols_target = ["PJME_MW"]

# Set data until 2016 as Training for the Model
df_train = df_PJME[:"2016-12-31"]
X = df_train[cols_train]
y = df_train[cols_target]

In [9]:
X

,Quarter,Month,Day,Hour,Day_of_Year,Weekday,Is_Weekend
Datetime,,,,,,,
2002-01-01 01:00:00,1,1,1,1,1,1,False
2002-01-01 02:00:00,1,1,1,2,1,1,False
2002-01-01 03:00:00,1,1,1,3,1,1,False
2002-01-01 04:00:00,1,1,1,4,1,1,False
2002-01-01 05:00:00,1,1,1,5,1,1,False
...,...,...,...,...,...,...,...
2016-12-31 19:00:00,4,12,31,19,366,5,True
2016-12-31 20:00:00,4,12,31,20,366,5,True
2016-12-31 21:00:00,4,12,31,21,366,5,True


In [10]:
y

,PJME_MW
Datetime,
2002-01-01 01:00:00,30393.0
2002-01-01 02:00:00,29265.0
2002-01-01 03:00:00,28357.0
2002-01-01 04:00:00,27899.0
2002-01-01 05:00:00,28057.0
...,...
2016-12-31 19:00:00,34478.0
2016-12-31 20:00:00,33328.0
2016-12-31 21:00:00,32197.0


In [11]:
model = xgb.XGBRegressor(booster='gbtree',n_estimators=1000)
model.fit(X, y)

,objective,'reg:squarederror'
,base_score,None
,booster,'gbtree'
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


## Save Model

In [14]:
with open('../app/model/model.pkl','wb') as f:
    pickle.dump(model, f)
